## 下面的程序将geohash需求数据重新转换为乡镇街道边界

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas
import pandas as pd
import geohash
import numpy as np
from pathlib import Path
from shapely.geometry import Polygon

E:\Anaconda\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_32528\4088112627.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migra

In [3]:
_AVG_EARTH_RADIUS_KM = 63710088 #km
def _polygon_area(lngs, lats, algorithm = 0, radius = _AVG_EARTH_RADIUS_KM):
    """
    https://github.com/anutkk/sphericalgeometry/blob/master/sphericalgeometry/highlevel.py#L145
    
    Computes area of spherical polygon, assuming spherical Earth. 
    Returns result in ratio of the sphere's area if the radius is specified. Otherwise, in the units of provided radius.
    lats and lons are in degrees.
    """
    #TODO: take into account geodesy (i.e. convert latitude to authalic sphere, use radius of authalic sphere instead of mean radius of spherical earth)
    import numpy as np
    lats = np.deg2rad(lats)
    lngs = np.deg2rad(lngs)

    if algorithm==0:
        # Line integral based on Green's Theorem, assumes spherical Earth
        from numpy import arctan2, cos, sin, sqrt, pi, power, append, diff

        #close polygon
        if lats[0]!=lats[-1]:
            lats = append(lats, lats[0])
            lngs = append(lngs, lngs[0])

        # Get colatitude (a measure of surface distance as an angle)
        a = sin(lats/2)**2 + cos(lats)* sin(lngs/2)**2
        colat = 2*arctan2( sqrt(a), sqrt(1-a) )

        #azimuth of each point in segment from the arbitrary origin
        az = arctan2(cos(lats) * sin(lngs), sin(lats)) % (2*pi)

        # Calculate step sizes
        # daz = diff(az) % (2*pi)
        daz = diff(az)
        daz = (daz + pi) % (2 * pi) - pi

        # Determine average surface distance for each step
        deltas=diff(colat)/2
        colat=colat[0:-1]+deltas

        # Integral over azimuth is 1-cos(colatitudes)
        integrands = (1-cos(colat)) * daz

        # Integrate and save the answer as a fraction of the unit sphere.
        # Note that the sum of the integrands will include a factor of 4pi.
        area = abs(sum(integrands))/(4*pi) # Could be area of inside or outside

        area = min(area,1-area)
        if radius is not None: #return in units of radius
            return area * 4*pi*radius**2
        else: #return in ratio of sphere total area
            return area
    elif algorithm==2:
        #L'Huilier Theorem, assumes spherical earth
        #see:
        # https://mathworld.wolfram.com/SphericalPolygon.html
        # https://web.archive.org/web/20160324191929/http://forum.worldwindcentral.com/showthread.php?20724-A-method-to-compute-the-area-of-a-spherical-polygon
        # https://github.com/spacetelescope/spherical_geometry/blob/master/spherical_geometry/polygon.py
        # https://github.com/tylerjereddy/spherical-SA-docker-demo/blob/master/docker_build/demonstration.py
        #TODO
        pass
    elif algorithm==3:
        #https://trs.jpl.nasa.gov/handle/2014/41271
        #TODO
        pass 
def area_estimation(tmp):
    from shapely import geometry
    if isinstance(tmp, geometry.polygon.Polygon):
        list_x, list_y = tmp.exterior.coords.xy
        list_x = list(list_x)
        list_y = list(list_y)
        return _polygon_area(list_y, list_x, algorithm = 0, radius = 6371008.8 )
    else:
        area = 0
        for geom in tmp.geoms:
            area += area_estimation(geom)

        return area  

In [4]:
def get_all_geohash(file):
    set_geohash=set()
    s=file['sZoneID']
    e=file['eZoneID']
    for i in range(len(s)):
        set_geohash.add(s.iloc[i])
        set_geohash.add(e.iloc[i])
    return set_geohash
def topolygon(zoneid):
    """
    """
    clat, clng, deltlat, deltlng = geohash.decode_exactly(zoneid)
    minx, miny, maxx, maxy = clng - deltlng, clat - deltlat, clng + deltlng, clat + deltlat
    return Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny), (minx, miny)])

In [5]:
# 原始数据确定
shpfolder="./浙江省_乡镇边界.shp"
city0="hangzhou"
city1="./"+city0
resultfolder=Path(city1)
datafolder=Path("./原始数据")

In [6]:
demand_data=pd.read_csv(datafolder/(city0+"_withcity.csv"))

In [7]:
shp=geopandas.read_file(shpfolder,encoding='gbk',crs='4326')

In [8]:
shp

gml_id  Name layer          code  grade  \
0       layer_township_pg.159   廊下镇    乡镇  310116107000      4   
1       layer_township_pg.160  金山卫镇    乡镇  310116109000      4   
2       layer_township_pg.163   枫泾镇    乡镇  310116102000      4   
3       layer_township_pg.164   吕巷镇    乡镇  310116105000      4   
4       layer_township_pg.192   金泽镇    乡镇  310118104000      4   
...                       ...   ...   ...           ...    ...   
1426  layer_township_pg.34553   佳阳乡    乡镇  350982202000      4   
1427  layer_township_pg.34557   叠石乡    乡镇  350982201000      4   
1428  layer_township_pg.34562   贯岭镇    乡镇  350982100000      4   
1429  layer_township_pg.34564   前岐镇    乡镇  350982101000      4   
1430  layer_township_pg.34567   沙埕镇    乡镇  350982102000      4   

                                               geometry  
0     MULTIPOLYGON (((121.19582 30.78543, 121.19582 ...  
1     MULTIPOLYGON (((121.26315 30.70874, 121.26297 ...  
2     MULTIPOLYGON (((121.11661 30.85222, 121.11650 ...  
3     MULTIPOLYGON (((121.13222 30.83222, 121.13210 ...  
4     MULTIPOLYGON (((120.97806 31.01831, 120.97806 ...  
...                                                 ...  
1426  MULTIPOLYGON (((120.37690 27.31998, 120.37693 ...  
1427  MULTIPOLYGON (((120.04172 27.34152, 120.04204 ...  
1428  MULTIPOLYGON (((120.19545 27.43085, 120.19536 ...  
1429  MULTIPOLYGON (((120.27830 27.39317, 120.27799 ...  
1430  MULTIPOLYGON (((120.83446 27.04579, 120.83438 ...  

[1431 rows x 6 columns]

In [9]:
demand_data=demand_data[demand_data['hour']>=6]
demand_data=demand_data[demand_data['hour']<=8]
demand_data=demand_data.groupby(['sZoneID','eZoneID']).agg({
    "demand":"sum","slng":"first","elng":"first","slat":"first","elat":"first"
}).reset_index()

In [10]:
demand_data

sZoneID eZoneID  demand        slng        elng       slat       elat
0       wthppd  wthr2b       1  118.460083  118.515015  29.369202  29.402161
1       wthppd  wthr3t       1  118.460083  118.547974  29.369202  29.429626
2       wthppd  wthr3v       1  118.460083  118.558960  29.369202  29.429626
3       wthppd  wthr3w       1  118.460083  118.547974  29.369202  29.435120
4       wthppd  wthr3x       1  118.460083  118.547974  29.369202  29.440613
...        ...     ...     ...         ...         ...        ...        ...
829812  wtmub3  wtmu8x       1  120.602417  120.613403  30.374451  30.363464
829813  wtmub8  wtmsxq       1  120.613403  120.558472  30.368958  30.357971
829814  wtmubc  wtmu9p       1  120.624390  120.635376  30.374451  30.363464
829815  wtmuc0  wtmsrv       1  120.635376  120.580444  30.368958  30.308533
829816  wtmuc0  wtmu9p       2  120.635376  120.635376  30.368958  30.363464

[829817 rows x 7 columns]

In [11]:
shp['jiedao_id']=range(shp.shape[0])
shp['jiedao_area']=shp['geometry'].apply(lambda v: area_estimation(v))
shp['p']=shp['geometry'].centroid
shp['Longitude']=shp['p'].x
shp['Latitude']=shp['p'].y
shp=shp.drop(columns=['gml_id'])
shp

E:\Anaconda\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_32528\3477712374.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shp['p']=shp['geometry'].centroid


Name layer          code  grade  \
0      廊下镇    乡镇  310116107000      4   
1     金山卫镇    乡镇  310116109000      4   
2      枫泾镇    乡镇  310116102000      4   
3      吕巷镇    乡镇  310116105000      4   
4      金泽镇    乡镇  310118104000      4   
...    ...   ...           ...    ...   
1426   佳阳乡    乡镇  350982202000      4   
1427   叠石乡    乡镇  350982201000      4   
1428   贯岭镇    乡镇  350982100000      4   
1429   前岐镇    乡镇  350982101000      4   
1430   沙埕镇    乡镇  350982102000      4   

                                               geometry  jiedao_id  \
0     MULTIPOLYGON (((121.19582 30.78543, 121.19582 ...          0   
1     MULTIPOLYGON (((121.26315 30.70874, 121.26297 ...          1   
2     MULTIPOLYGON (((121.11661 30.85222, 121.11650 ...          2   
3     MULTIPOLYGON (((121.13222 30.83222, 121.13210 ...          3   
4     MULTIPOLYGON (((120.97806 31.01831, 120.97806 ...          4   
...                                                 ...        ...   
1426  MULTIPOLYGON (((120.37690 27.31998, 120.37693 ...       1426   
1427  MULTIPOLYGON (((120.04172 27.34152, 120.04204 ...       1427   
1428  MULTIPOLYGON (((120.19545 27.43085, 120.19536 ...       1428   
1429  MULTIPOLYGON (((120.27830 27.39317, 120.27799 ...       1429   
1430  MULTIPOLYGON (((120.83446 27.04579, 120.83438 ...       1430   

        jiedao_area                           p   Longitude   Latitude  
0      27153.973040  POINT (121.16352 30.78386)  121.163518  30.783860  
1       2796.554372  POINT (121.24380 30.74945)  121.243803  30.749452  
2      74736.148370  POINT (121.03241 30.84720)  121.032414  30.847204  
3       4656.239307  POINT (121.12603 30.83352)  121.126034  30.833523  
4      93874.435786  POINT (120.93150 31.01885)  120.931500  31.018849  
...             ...                         ...         ...        ...  
1426  127092.963555  POINT (120.40061 27.27577)  120.400606  27.275769  
1427  402519.826255  POINT (120.11911 27.40000)  120.119108  27.400000  
1428  143175.495576  POINT (120.23337 27.42657)  120.233367  27.426574  
1429  227329.855966  POINT (120.32429 27.38356)  120.324289  27.383563  
1430  111251.936947  POINT (120.52724 27.15382)  120.527243  27.153818  

[1431 rows x 10 columns]

In [12]:
all_geohash=list(get_all_geohash(demand_data))
all_geohash=pd.DataFrame(all_geohash,columns=['geohash_id'])
gdf_geohash=geopandas.GeoDataFrame(all_geohash,crs=4326,geometry=[topolygon(zoneid) for 
                                                                 zoneid in all_geohash['geohash_id']])
gdf_geohash['geohash_area']=gdf_geohash['geometry'].apply(lambda v:area_estimation(v))

E:\Anaconda\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [13]:
gdf_geohash

geohash_id                                           geometry  \
0         wtmskr  POLYGON ((120.42114 30.31677, 120.42114 30.322...   
1         wtk8tc  POLYGON ((119.08081 29.62463, 119.08081 29.630...   
2         wtmd3g  POLYGON ((120.31128 29.94324, 120.31128 29.948...   
3         wtmgbf  POLYGON ((120.61890 30.20142, 120.61890 30.206...   
4         wtm5ck  POLYGON ((119.58618 30.21240, 119.58618 30.217...   
...          ...                                                ...   
13604     wtmese  POLYGON ((120.43213 30.16296, 120.43213 30.168...   
13605     wtmh4e  POLYGON ((119.64111 30.25085, 119.64111 30.256...   
13606     wtmh0k  POLYGON ((119.54224 30.25635, 119.54224 30.261...   
13607     wtmt2n  POLYGON ((120.23438 30.48706, 120.23438 30.492...   
13608     wtku7p  POLYGON ((119.31152 30.31677, 119.31152 30.322...   

        geohash_area  
0      377893.754787  
1      362740.103265  
2      376659.320759  
3      380112.231749  
4      368477.536437  
...              ...  
13604  378017.121864  
13605  369099.451114  
13606  367979.760810  
13607  375794.392605  
13608  365362.894054  

[13609 rows x 3 columns]

In [14]:
gdf_join=gdf_geohash.overlay(shp[['jiedao_id','geometry','Longitude','Latitude']],how='intersection')
gdf_join['inter_area']=gdf_join['geometry'].apply(lambda v:area_estimation(v))
gdf_join['geohash_ratio']=gdf_join['inter_area']/gdf_join['geohash_area']
df_inter=pd.DataFrame(gdf_join[['geohash_id','jiedao_id','geohash_ratio','Longitude','Latitude']])

E:\Anaconda\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [15]:
demand_data=demand_data.rename(columns={"sZoneID":"sgeohash","eZoneID":"egeohash"})
df_inter.columns=['sgeohash','sjiedao','sratio','slongitude','slatitude']
demand_data=pd.merge(demand_data,df_inter,on=['sgeohash'])
df_inter.columns=['egeohash','ejiedao','eratio','elongitude','elatitude']
demand_data=pd.merge(demand_data,df_inter,on=['egeohash'])
df_inter.columns=['geohash_id','jiedao_id','geohash_ratio','longitude','latitude']

In [16]:
demand_data

sgeohash egeohash  demand        slng        elng       slat  \
0         wthppd   wthr2b       1  118.460083  118.515015  29.369202   
1         wthppd   wthr2b       1  118.460083  118.515015  29.369202   
2         wthr0m   wthr2b       1  118.493042  118.515015  29.385681   
3         wthr0x   wthr2b       4  118.504028  118.515015  29.396667   
4         wthr2n   wthr2b       1  118.482056  118.515015  29.435120   
...          ...      ...     ...         ...         ...        ...   
1978497   wtmu9e   wtmu9s       1  120.657349  120.657349  30.341492   
1978498   wtmu9t   wtmu9s       1  120.657349  120.657349  30.352478   
1978499   wtmu9h   wtmu8v       1  120.635376  120.624390  30.346985   
1978500   wtmu9s   wtmu9e       3  120.657349  120.657349  30.346985   
1978501   wtmu9s   wtmu9t       1  120.657349  120.657349  30.346985   

              elat  sjiedao    sratio  slongitude  slatitude  ejiedao  eratio  \
0        29.402161      726  0.999969  118.563538  29.421156      726     1.0   
1        29.402161     1177  0.000032  118.481252  29.343462      726     1.0   
2        29.402161      726  1.000000  118.563538  29.421156      726     1.0   
3        29.402161      726  1.000000  118.563538  29.421156      726     1.0   
4        29.402161      729  1.000000  118.435544  29.454622      726     1.0   
...            ...      ...       ...         ...        ...      ...     ...   
1978497  30.346985      780  1.000000  120.611247  30.325751      780     1.0   
1978498  30.346985      780  1.000000  120.611247  30.325751      780     1.0   
1978499  30.352478      780  1.000000  120.611247  30.325751      780     1.0   
1978500  30.341492      780  1.000000  120.611247  30.325751      780     1.0   
1978501  30.352478      780  1.000000  120.611247  30.325751      780     1.0   

         elongitude  elatitude  
0        118.563538  29.421156  
1        118.563538  29.421156  
2        118.563538  29.421156  
3        118.563538  29.421156  
4        118.563538  29.421156  
...             ...        ...  
1978497  120.611247  30.325751  
1978498  120.611247  30.325751  
1978499  120.611247  30.325751  
1978500  120.611247  30.325751  
1978501  120.611247  30.325751  

[1978502 rows x 15 columns]

In [17]:
demand_data['inter_demand']=demand_data['sratio']*demand_data['eratio']*demand_data['demand']
demand_data=demand_data.groupby(['sjiedao','ejiedao']).agg({
    "slongitude":"first","slatitude":"first","elongitude":"first","elatitude":"first",
    "inter_demand":"sum"
}).reset_index()
demand_data.columns=['sZoneID','eZoneID','slng','slat','elng','elat','demand']

In [18]:
demand_data=demand_data[demand_data['demand']>1]

In [19]:
demand_data

sZoneID  eZoneID        slng       slat        elng       elat  \
3          402      402  120.344515  30.507799  120.344515  30.507799   
16         402      635  120.344515  30.507799  120.313744  30.446881   
18         402      639  120.344515  30.507799  120.258745  30.437242   
19         402      642  120.344515  30.507799  120.290210  30.488228   
20         402      644  120.344515  30.507799  120.306060  30.391267   
...        ...      ...         ...        ...         ...        ...   
22849     1374      702  119.793205  29.524867  119.812876  29.836723   
22852     1374      707  119.793205  29.524867  119.704206  29.768015   
22853     1374      709  119.793205  29.524867  119.602229  29.863949   
22856     1374      713  119.793205  29.524867  119.799263  29.749147   
22857     1374      714  119.793205  29.524867  119.684410  29.684392   

          demand  
3       1.190271  
16      3.941354  
18      3.083593  
19     39.326809  
20      1.242126  
...          ...  
22849   1.133376  
22852   3.577034  
22853   1.874224  
22856   2.002654  
22857   3.294819  

[16284 rows x 7 columns]

In [20]:
szone=demand_data.groupby(['sZoneID']).agg({"slng":"first","slat":"first"}).reset_index()
ezone=demand_data.groupby(['eZoneID']).agg({"elng":"first","elat":"first"}).reset_index()
szone.columns=['ZoneID','lng','lat']
ezone.columns=['ZoneID','lng','lat']
zone=pd.concat([szone,ezone])
zone=zone.groupby('ZoneID').agg({"lng":"first","lat":"first"}).reset_index()
zone

ZoneID         lng        lat
0       402  120.344515  30.507799
1       415  120.431667  30.421093
2       421  120.367124  30.432714
3       600  120.158617  30.241487
4       601  120.191967  30.236616
..      ...         ...        ...
216    1296  119.392448  29.361163
217    1363  119.749815  29.455938
218    1364  119.821558  29.569834
219    1369  119.912715  29.639245
220    1374  119.793205  29.524867

[221 rows x 3 columns]

In [21]:
zone_info=zone.values
all_1=np.tile(zone_info,zone_info.shape[0]).reshape(-1,zone_info.shape[1])
all_2=np.tile(zone_info,(zone_info.shape[0],1))
all_od=pd.DataFrame(np.hstack([all_1,all_2]),columns=['sZoneID','slng','slat','eZoneID','elng','elat'])
all_od

sZoneID        slng       slat  eZoneID        elng       elat
0        402.0  120.344515  30.507799    402.0  120.344515  30.507799
1        402.0  120.344515  30.507799    415.0  120.431667  30.421093
2        402.0  120.344515  30.507799    421.0  120.367124  30.432714
3        402.0  120.344515  30.507799    600.0  120.158617  30.241487
4        402.0  120.344515  30.507799    601.0  120.191967  30.236616
...        ...         ...        ...      ...         ...        ...
48836   1374.0  119.793205  29.524867   1296.0  119.392448  29.361163
48837   1374.0  119.793205  29.524867   1363.0  119.749815  29.455938
48838   1374.0  119.793205  29.524867   1364.0  119.821558  29.569834
48839   1374.0  119.793205  29.524867   1369.0  119.912715  29.639245
48840   1374.0  119.793205  29.524867   1374.0  119.793205  29.524867

[48841 rows x 6 columns]

In [23]:
all_od.to_csv(resultfolder/"all_od_pair_info_final.csv",index=None)
zone.to_csv(resultfolder/"zip_change.csv",index=None)
demand_data.to_csv(resultfolder/"demand_with_duration_final.csv",index=None)